In [1]:
# Imports helper functions
from kaggle_environments.envs.halite.helpers import *
import numpy as np

# Returns best direction to move from one position (fromPos) to another (toPos)
# Example: If I'm at pos 0 and want to get to pos 55, which direction should I choose?
def getDirTo(fromPos, toPos, size):
    fromX, fromY = divmod(fromPos[0],size), divmod(fromPos[1],size)
    toX, toY = divmod(toPos[0],size), divmod(toPos[1],size)
    if fromY < toY: return ShipAction.NORTH
    if fromY > toY: return ShipAction.SOUTH
    if fromX < toX: return ShipAction.EAST
    if fromX > toX: return ShipAction.WEST

# Directions a ship can move
directions = [ShipAction.NORTH, ShipAction.EAST, ShipAction.SOUTH, ShipAction.WEST]

# Will keep track of whether a ship is collecting halite or carrying cargo to a shipyard
ship_states = {}


def agent(obs, config):
    size = config.size
    board = Board(obs, config)
    me = board.current_player

    # If there are few ships, use first shipyard to spawn a ship.
    if len(me.ships) < 3 and len(me.shipyards) > 0:
        me.shipyards[0].next_action = ShipyardAction.SPAWN

    # If there are no shipyards, convert first ship into shipyard.
    if len(me.shipyards) == 0 and len(me.ships) > 0:
        me.ships[0].next_action = ShipAction.CONVERT
    
    for ship in me.ships:
        if ship.next_action == None:
            
            ### Part 1: Set the ship's state 
            if ship.halite < 200: # If cargo is too low, collect halite
                ship_states[ship.id] = "COLLECT"
            if ship.halite > 500: # If cargo gets very big, deposit halite
                ship_states[ship.id] = "DEPOSIT"
                
            ### Part 2: Use the ship's state to select an action
            if ship_states[ship.id] == "COLLECT":
                # If halite at current location running low, 
                # move to the adjacent square containing the most halite
                if ship.cell.halite < 100:
                    neighbors = [ship.cell.north.halite, ship.cell.east.halite, 
                                 ship.cell.south.halite, ship.cell.west.halite]
                    best = max(range(len(neighbors)), key=neighbors.__getitem__)

                    ship.next_action = directions[best]
            if ship_states[ship.id] == "DEPOSIT":
                # Move towards shipyard to deposit cargo
                direction = getDirTo(ship.position, me.shipyards[0].position, size)
                if direction: ship.next_action = direction
                
    return me.next_actions

In [106]:
board_size = 7
agent_count = 1
environment = make("halite", configuration={"size": board_size, "startingHalite": 1000})
environment.reset(agent_count)
board = Board(environment.state[0].observation, environment.configuration)

In [97]:
def get_min_x(current_x, board_size):
    """Return the minimum x-coordinate to scan for halite.
    
    Max scan range is 2 cells."""
    MAX_SCAN_RANGE = 2
    min_x = current_x - MAX_SCAN_RANGE
    # Transform to eliminate negative values.
    min_x = (min_x + board_size) % board_size
    return min_x
assert get_min_x(2, 5) == 0
assert get_min_x(0, 5) == 3
assert get_min_x(4, 5) == 2
assert get_min_x(1, 5) == 4

In [113]:
def get_min_y(current_y, board_size):
    """Return the minimum y-coordinate to scan for halite.
    
    Max scan range is 2 cells."""
    MAX_SCAN_RANGE = 2
    min_y = current_y - MAX_SCAN_RANGE
    # Transform to eliminate negative values.
    min_y = (min_y + board_size) % board_size
    return min_y
assert get_min_y(2, 5) == 0
assert get_min_y(0, 5) == 3
assert get_min_y(4, 5) == 2
assert get_min_y(1, 5) == 4

In [100]:
def get_max_x(current_x, board_size):
    """Return the maximum x-coordinate to scan for halite.
    
    Max scan range is 2 cells."""
    MAX_SCAN_RANGE = 2
    max_x = current_x + MAX_SCAN_RANGE
    max_x = (max_x + board_size) % board_size
    return max_x 
assert get_max_x(2, 5) == 4
assert get_max_x(4, 5) == 1
assert get_max_x(1, 5) == 3

In [103]:
def get_max_y(current_y, board_size):
    """Return the maximum x-coordinate to scan for halite.
    
    Max scan range is 2 cells."""
    MAX_SCAN_RANGE = 2
    max_y = current_y + MAX_SCAN_RANGE
    max_y = (max_y + board_size) % board_size
    return max_y 
assert get_max_y(2, 5) == 4
assert get_max_y(4, 5) == 1
assert get_max_y(1, 5) == 3

In [96]:
def get_x_list(min_x, max_x, board_size):
    if max_x < min_x:
        max_x = max_x + board_size
    x_array = np.array(range(min_x , max_x + 1))
    x_list = list(x_array % board_size)
    return x_list
assert get_x_list(1, 2, 5) == [1, 2]
assert get_x_list(4, 0, 5) == [4, 0]
assert get_x_list(0, 4, 5) == [0, 1, 2, 3, 4]
assert get_x_list(1, 0, 5) == [1, 2, 3, 4, 0]

In [102]:
def get_y_list(min_y, max_y, board_size):
    if max_y < min_y:
        max_y = max_y + board_size
    y_array = np.array(range(min_y , max_y + 1))
    y_list = list(y_array % board_size)
    return y_list
assert get_y_list(1, 2, 5) == [1, 2]
assert get_y_list(4, 0, 5) == [4, 0]
assert get_y_list(0, 4, 5) == [0, 1, 2, 3, 4]
assert get_y_list(1, 0, 5) == [1, 2, 3, 4, 0]

In [110]:
def get_target_cells(x_list, y_list, board):
    
    halite_list = []
    for x in x_list:
        for y in y_list:
            halite_list.append([(x, y), board.cell[x, y].halite])
    return halite_list

In [139]:
a = np.array(board.observation['halite'])

In [140]:
a

array([10,  0, 15, 51, 15,  0, 10, 40, 60,  0,  0,  0, 60, 40,  0,  0,  1,
       26,  1,  0,  0, 21,  0,  8, 13,  8,  0, 21,  0,  0,  1, 26,  1,  0,
        0, 40, 60,  0,  0,  0, 60, 40, 10,  0, 15, 51, 15,  0, 10])

In [159]:
a = a.reshape(board_size, board_size)

In [145]:
board.players[0].ships

In [152]:
b = board.players[0].ships[0].cell.position

In [154]:
list(b)

[3, 3]

In [213]:
def play_game(board):
    board = get_move(board)
    return play_game(board)

def get_move(board):
    print(board)
    actions = {'w': ShipAction.NORTH,
              's': ShipAction.SOUTH,
              'a': ShipAction.WEST,
              'd': ShipAction.EAST,
              ' ': ShipAction.CONVERT, 
              'x': ShipyardAction.SPAWN}
    for ship in board.current_player.ships:
        capture_action = input("Action for ship at {}:".format(ship.position))
        ship.next_action = actions[capture_action]
    for sy in board.current_player.shipyards:
        capture_action = input("Action for shipyard at {}:".format(sy.position))
        sy.next_action = actions[capture_action]
        
    print(board.next())
    return board.next()

In [ ]:
def get_point(line_index, board_size):
    """Get the coordinates of a cell. Inputs are the line index and board size.
    
    Halite amounts are given in 1-d list format. Transform that so the 
    coordinates can be used."""
    point_address = (line_index % board_size, line_index // board_size)
    return point_address
#assert get_point(1, 5) == (1, 0)
#assert get_point(6, 5) == "(1, 1)"
#assert get_point(24, 5) == "(4, 4)"
#assert get_point(0, 5) == "(0, 0)"
#assert get_point(440, 21) == "(20, 20)"

def get_rich_cells(board):
    """Return the cell index with maximum halite"""
    rich_cells = []
    for i in range(len(board.observation['halite'])):
        if board.observation['halite'][i] == max(board.observation['halite']):
            rich_cells.append(i)
    return rich_cells

In [13]:
environment.run([agent])
environment.render(mode="ipython", width=800, height=600)